In [1]:
from datetime import datetime
import tensorflow as tf
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import PurePosixPath
now = str(datetime.now().date())

In [2]:
# import config 
import configs.config
importlib.reload(configs.config)
from configs.config import CFG
from utils import config
config_train = config.Config.from_json(CFG)

## Create Dataset

In [3]:
# create Train Series and Dataset 
import dataloader.dataloader
importlib.reload(dataloader.dataloader)
from dataloader.dataloader import Train_Series
try: 
    del series
except:
    pass
from dataloader.dataloader import Test_Series
try: 
    del test_series
except:
    pass



In [4]:
# define train
epochs = config_train.train.epoches
batch_size = config_train.train.batch_size
stride = config_train.train.stride

shuffle_buffer_size = config_train.train.shuffle_buffer_size

#define model
num_classes = config_train.model.output

In [5]:
from dataloader.dataloader import Serie
class TestSerie(Serie):

    def __init__(self, serie_id, serie_path, serie_events,**kwargs):
        super().__init__(serie_id, serie_path, serie_events,**kwargs)
        self.window_size = None

    def _find_window_size(self,x,preferred_window_size =10):
        def get_divisors(n):
            divisors = []
            for i in range(1, int(n / 2) + 1):
                if n % i == 0:
                    divisors.append(i)
            return divisors
        print(x.shape)
        length =x.shape[-2]
        if (length % preferred_window_size) == 0:
            return preferred_window_size

        else: 
            divisors = get_divisors(length)
            divisors_amount = len(divisors)
            for i in range(len(divisors)): 
                if length // divisors[i] < preferred_window_size : 
                    return length//divisors[i]

                elif i == divisors_amount-1 :
                    return length//divisors[i]
                else:
                    continue
    def _windowed_trainset(self,series,  window_size,   batch_size,stride=1,drop_remainder=True):
        """Generates dataset windows

        Args:
        series (array of float) - contains the values of the time series
        labels - contains 
        window_size (int) - the number of time steps to average
        batch_size (int) - the batch size
        shuffle_buffer(int) - buffer size to use for the shuffle method

        Returns:
        dataset (TF Dataset) - TF Dataset containing time windows
        """
        dataset = series.reshape(-1,series.shape[-1])
        # Generate a TF Dataset from the series values
        dataset = tf.data.Dataset.from_tensor_slices(dataset)
        # Window the data but only take those with the specified size
        dataset = dataset.window(window_size+1, shift = stride, drop_remainder=drop_remainder,)
        # Flatten the windows by putting its elements in a single batch
        dataset = dataset.flat_map(lambda window: window.batch(window_size+1))    
        dataset = dataset.map(lambda window: window[:-1,:])        
        # Create batches of windows
        dataset = dataset.batch(batch_size).prefetch(1)
        return dataset
    
    def _transform(self,x,std,epsilon):
        return (x)/(epsilon+np.abs(std))
    
    def create_slices(self, time_window, drop_columns,std,epsilon,batch_size):
        #super().create_slices(time_window,drop_columns,self.serie)
        data = self.serie.drop(columns =drop_columns).to_numpy()
        data = self._transform(data,std,epsilon)
        print(time_window)
        self.window_size = self._find_window_size(data,preferred_window_size =time_window)
        self.slices = self._windowed_trainset(data,self.window_size,batch_size)
        
    def create_events(self, y_pred: np.array):
        self.serie_events, raw_prediction = Serie.create_events(self,y_pred)
        return self.serie_events , raw_prediction
    
    def get_correct_slices(self):

        return self.slices

In [6]:
from dataloader.dataloader import Series
class Test_Series(Series):
    def __init__(self,data,paths):
        super().__init__(data,paths)
        self.steps_window, self.valid_steps = super().set_ranges(self.data.test_slice_length)
    
    # def set_ranges(self):
    #     steps_window, valid_steps = super().set_ranges()
    #     return steps_window, valid_steps

    def createSerie(self,serie_id):
            if serie_id in self.series_names:
                ## TODO - place for logger warning
                return self.series[serie_id]
            serie_filename = serie_id+"."+self.data.series_format
            serie_path = self.paths.test_series / serie_filename
            serie = TestSerie(serie_id,serie_path,
                              None,
                              gradient_difference = self.data.gradient_difference, 
                              moving_average_gradient_samples = self.data.moving_average_gradient_samples, 
                              moving_average_enmo_samples = self.data.moving_average_enmo_samples)
            self.series[serie_id] = serie
            self.series_names.add(serie_id)
            self.series_ids.append(serie_id)
            return serie

    def createSeries(self,std,epsilon,batch_size):
        # extract all series ids in test series dataset
        p = self.paths.test_series.glob(f'*.{self.data.series_format}')
        series_files = [PurePosixPath(x).stem for x in p if x.is_file()]
        for serie_id in series_files:   
            serie = self.createSerie(serie_id)
            #serie.create_slices(self.steps_window,["step","timestamp"])
            serie.create_slices(time_window=self.steps_window,
                                drop_columns=["step","timestamp"],
                                std=std,
                                epsilon=epsilon,
                                batch_size=batch_size)
    def createSeriesEvents(self,events : dict):
        """
        input: 
        events - dictionary with serie_id as key 
                and values equal to numpy array:
                 - column 1 - segmentation mask
                 - column 2 - score   """
        first_event = False
        series_event = None
        for serie_id in self.series.keys():
            serie_event =self.series[serie_id].create_events(events[serie_id])
            if first_event:
                series_event = serie_event
                first_event = True
                continue
            else:
                series_event = pd.concat([series_event,serie_event])
        
        series_event["row_id"] = range(0,len(series_event))
        self.df_events = series_event
        return self.df_events

In [7]:
# create Train Series and Dataset 
ds_std = np.array([42.12841457 , 0.06510425 , 0.05023697 ,39.93350993,  5.91329642])
epsilon = 0.000000000000001 
test_series = Test_Series(config_train.data,config_train.paths)
test_series.createSeries(std=ds_std,epsilon=epsilon,batch_size=config_train.train.batch_size)



10
(150, 5)
10
(150, 5)
10
(150, 5)


In [8]:
try:
    del config_VAL, CFG_VAL,val_series,config,config
except:
    pass
CFG_VAL = {
      "paths":{
        "train_events": "./dataloader/datasets/v1/train-events/train_events.csv",
        "train_series": "./dataloader/datasets/v1/train-series/",
        "test_series":"./dataloader/datasets/v1/train-series/"
      },

      "data": {
        "series_format":   "csv",
        "record_interval": 5,  # every step recorded in interval of x [s]
        "slice_length" :75/360, #8/36, # in [h] 75/360 previously = 150 steps
        "test_slice_length":5/360, # should be equal to slice_length if windowed dataset is not used
        "valid_range_ifNan" : 1, # range of time to consider as valid if Nan was detected (before/after) in[h]
        "clean_data" : True,
        "limit_slices" : True,
        "limit_window": 3,
        "feature_engineering" : True,
        "gradient_difference" : 1,
        "moving_average_gradient_samples" : 3,
        "moving_average_enmo_samples" : 10,
        "shuffle_Dataset" : False,
        "normalize" : True
    },
    "train": {
        "batch_size": 64,
        "shuffle_buffer_size": 1500,
        "stride" : 1,
        "epoches": 30,
    },
    "model": {
        "output": 2
    }
}

from utils import config

config_VAL = config.Config.from_json(CFG_VAL)
val_series = Test_Series(config_VAL.data,config_VAL.paths)
val_series.createSeries(std=ds_std,epsilon=epsilon,batch_size=config_VAL.train.batch_size)



10
(724140, 5)
10
(637560, 5)
10
(400860, 5)
10
(388260, 5)
10
(115380, 5)
10
(603900, 5)
10
(534960, 5)
10
(583200, 5)
10
(455040, 5)
10
(601020, 5)
10
(588240, 5)
10
(241560, 5)
10
(354420, 5)
10
(401940, 5)
10
(416880, 5)
10
(103500, 5)
10
(594540, 5)
10
(409680, 5)
10
(435420, 5)
10
(392940, 5)
10
(404280, 5)
10
(388080, 5)
10
(532800, 5)
10
(417780, 5)
10
(387720, 5)
10
(379980, 5)
10
(726300, 5)
10
(410400, 5)
10
(407880, 5)
10
(388080, 5)
10
(397260, 5)
10
(136620, 5)
10
(358920, 5)
10
(389880, 5)
10
(404280, 5)
10
(585540, 5)
10
(603180, 5)
10
(412560, 5)
10
(759240, 5)
10
(687780, 5)
10
(426240, 5)
10
(390240, 5)
10
(385380, 5)


In [9]:
config_VAL.paths is config_train.paths

False

In [10]:
config_VAL.paths.test_series

WindowsPath('dataloader/datasets/v1/train-series')

## Build the Model

In [11]:
num_features = 5
# Build the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: x,input_shape=[None,num_features]),
  #tf.keras.layers.Conv1D(32,1,padding="same",input_shape=[None,num_features]),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.2,)),
  #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True,dropout=0.2)),
  tf.keras.layers.Dense(2,activation ='softmax'),
  #tf.keras.layers.Dense(num_classes,activation ='sigmoid')   
])

# Set the learning rate
learning_rate = 0.0158
epochs = 30


# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Set the training parameters
model.compile(
    #loss=tf.keras.losses.Huber(), optimizer=optimizer)
    optimizer = optimizer,
    loss = "binary_crossentropy",
    metrics=['binary_accuracy'])
    # loss=tf.keras.losses.CategoricalCrossentropy(),
    # metrics=['categorical_accuracy'])

    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, None, 5)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               9728      
 l)                                                              
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 9,858
Trainable params: 9,858
Non-trainable params: 0
_________________________________________________________________


In [12]:
model_name = f"lstm_1x32_lr_{learning_rate}"
epoch = epochs
date = "2023-12-03"
model_name
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "model/weights/cp-{model_name}-{epoch:04d}_{date}.ckpt"
# Loads the weights
model.load_weights(checkpoint_path.format(epoch =epoch, date =date,model_name=model_name))

In [17]:
def predict(model,serie):
    x = serie.get_correct_slices()
    print(x)
    y_pred = None

    try:
        y_pred = model.predict(x)
        y_pred = np.expand_dims(y_pred, axis=0)
        #y_pred = y_pred.reshape(1,-1, y_pred.shape[-1])
    except ValueError:
        print(x.shape)

    serie_events  = Serie.create_events(serie,y_pred,only_top_score=False)  

    return serie_events, y_pred

In [18]:
raw_predictions = {}
series_pred = np.array([])
y_pred = None
pred_start = False
for serie_id, serie in test_series.series.items():
    serie_events, y = predict(model,serie)
    serie_events['step'] = serie_events['step'] + serie.window_size
    #print(serie_id)
    #print(serie_events.shape)
    if pred_start == False:
        series_pred = y
        
        y_pred = serie_events
        pred_start = True
    else:
        series_pred = np.concatenate((series_pred, y))
        y_pred = pd.concat([y_pred, serie_events])

y_pred

<_PrefetchDataset element_spec=TensorSpec(shape=(None, None, 5), dtype=tf.float64, name=None)>
3/3 [==============================] - 0s 11ms/step


038441c925bb 8
038441c925bb 23
<_PrefetchDataset element_spec=TensorSpec(shape=(None, None, 5), dtype=tf.float64, name=None)>
3/3 [==============================] - 0s 13ms/step
03d92c9f6f8a 22
<_PrefetchDataset element_spec=TensorSpec(shape=(None, None, 5), dtype=tf.float64, name=None)>
3/3 [==============================] - 0s 8ms/step


,series_id,step,event,score
0,038441c925bb,18,wakeup,0.7
1,038441c925bb,33,onset,0.6
0,03d92c9f6f8a,32,onset,0.6


In [19]:
raw_predictions = {}
series_pred = np.array([])
y_pred = None
pred_start = False
for serie_id, serie in val_series.series.items():
    serie_events, y = predict(model,serie)
    serie_events['step'] = serie_events['step'] + serie.window_size
    #print(serie_id)
    #print(serie_events.shape)
    if pred_start == False:
        series_pred = y
        
        y_pred = serie_events
        pred_start = True
    else:
        print(serie_id)
        print(serie_events.shape)
        print(series_pred.shape)
        #series_pred = np.concatenate((series_pred, y))
        y_pred = pd.concat([y_pred, serie_events])

y_pred

<_PrefetchDataset element_spec=TensorSpec(shape=(None, None, 5), dtype=tf.float64, name=None)>
      1/Unknown - 0s 26ms/step

11315/11315 [==============================] - 56s 5ms/step
03d92c9f6f8a 22
03d92c9f6f8a 161
03d92c9f6f8a 732
03d92c9f6f8a 737
03d92c9f6f8a 805
03d92c9f6f8a 806
03d92c9f6f8a 815
03d92c9f6f8a 817
03d92c9f6f8a 839
03d92c9f6f8a 842
03d92c9f6f8a 1578
03d92c9f6f8a 1580
03d92c9f6f8a 1582
03d92c9f6f8a 1583
03d92c9f6f8a 1734
03d92c9f6f8a 1736
03d92c9f6f8a 1749
03d92c9f6f8a 1752
03d92c9f6f8a 3390
03d92c9f6f8a 3403
03d92c9f6f8a 3407
03d92c9f6f8a 3410
03d92c9f6f8a 3417
03d92c9f6f8a 3420
03d92c9f6f8a 3422
03d92c9f6f8a 3508
03d92c9f6f8a 3513
03d92c9f6f8a 3516
03d92c9f6f8a 3517
03d92c9f6f8a 3544
03d92c9f6f8a 3553
03d92c9f6f8a 3576
03d92c9f6f8a 3593
03d92c9f6f8a 3602
03d92c9f6f8a 3611
03d92c9f6f8a 3628
03d92c9f6f8a 3635
03d92c9f6f8a 3648
03d92c9f6f8a 3659
03d92c9f6f8a 3687
03d92c9f6f8a 3690
03d92c9f6f8a 3748
03d92c9f6f8a 3767
03d92c9f6f8a 3768
03d92c9f6f8a 3771
03d92c9f6f8a 3801
03d92c9f6f8a 3981
03d92c9f6f8a 4076
03d92c9f6f8a 4083
03d92c9f6f8a 4111
03d92c9f6f8a 4122
03d92c9f6f8a 4251
03d92c9f6f8a 42

KeyboardInterrupt: 